<a href="https://colab.research.google.com/github/Venchislav/Neural-Networks/blob/main/CustomFittingOnLowerLevel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.datasets import mnist

# **DATA LOAD**

In [4]:
def get_mnist_model():
  inputs = keras.Input(shape=(28 * 28,))
  features = layers.Dense(512, activation='relu')(inputs)
  features = layers.Dropout(0.5)(features)
  outputs = layers.Dense(10, activation='softmax')(features)

  model = keras.Model(inputs, outputs)
  return model


(images, labels), (test_images, test_labels) = mnist.load_data()
images = images.reshape((60_000, 28 * 28)).astype('float32') / 255.0
test_images = test_images.reshape((10_000, 28 * 28)).astype('float32') / 255.0

train_images, val_images = images[10_000:], images[:10_000]
train_labels, val_labels = labels[10_000:], labels[:10_000]


11490434/11490434 [==============================] - 0s 0us/step


In [5]:
model = get_mnist_model()

In [7]:
loss_fn = keras.losses.SparseCategoricalCrossentropy()
optimizer = keras.optimizers.RMSprop()
metrics = keras.metrics.SparseCategoricalAccuracy()
loss_tracking_metric = keras.metrics.Mean()

In [8]:
def train_step(input, targets):
  with tf.GradientTape() as tape:
    prediction = model(input, training=True)
    loss = loss_fn(targets, prediction)
  gradients = tape.gradient(loss, model.trainable_weights)
  optimizer.apply_gradients(zip(model.trainable_weights, gradients))

  logs = {}
  for metric in metrics:
    metric.update_state(targets, prediction)
    logs[metric.name] = metric.result()

  loss_tracking_metric.update_state(loss)
  logs['loss'] = loss_tracking_metric.result()
  return logs

In [9]:
def reset_metrics():
  for metric in metrics:
    metric.reset_state()
  loss_tracking_metric.reset_state()

In [11]:
training_dataset = tf.data.Dataset.from_tensor_slices(
    (train_images, train_labels)
)
training_dataset = training_dataset.batch(32)
epochs = 3

for epoch in range(epochs):
  reset_metrics()
  for inputs_batch, targets_batch in training_dataset:
    logs = train_step(inputs_batch, targets_batch)
  print(f'results at the end of {epoch} epoch')
  for key, value in logs.items():
    print(f'...{key}: {value}')

TypeError: 'SparseCategoricalAccuracy' object is not iterable